In [2]:
%%capture
%run 01_transaction.ipynb

In [3]:
from IPython.display import Image
from random import randint as ri

Create random txs.

In [17]:
def rtxs(n): return [TX(rh(),rh(),ri(1,9),ri(1,9)/10,0) for _ in range(n)] 
print(txs2str(rtxs(2)))

time:	Tue Mar 30 16:01:31 2021
from:	🔩 0xc3165bcdd3...a69
to:	💹 0x53b985d9cf...319
value:	1 ether
fee:	0.9 ether
nonce:	0
hash:	🔰 0xca77cc3bf9...021
signed:	false

time:	Tue Mar 30 16:01:31 2021
from:	🕍 0xe73a0d829d...bb9
to:	📳 0x8d81df1b52...638
value:	9 ether
fee:	0.2 ether
nonce:	0
hash:	🔅 0x9f3e90118f...25a
signed:	false



## Merkle Tree

A binary tree in which every leaf node is a hash of a specific piece of data, in our case a tx. 

In a system like Bitcoin or Ethereum the merkle root is included in the block header, not the list of txs itself. This makes it very efficient to test if a specific tx is inclued in the tree.

For more in depth [here](https://www.youtube.com/watch?v=3giNelTfeAk).

<div>
<img src="https://changelly.com/blog/wp-content/uploads/2020/01/Merkle-Tree-1.png" width="500"/>
</div>


`MerkleTree` is a wrapper around the `mt` dictionary which stores the hashes for each height in the tree.

In [18]:
class MerkleTree:
    def __init__(self, txs):
        self.mt = {}
        leafs = [tx.hash for tx in txs]
        if len(leafs)%2!=0: leafs.append(sha('0x0'))
        self.mt['1'] = leafs
        self.merkle(leafs)
        
    def merkle(self, leafs):
        parents  = []
        while len(leafs) > 1:
            for i in range(0, len(leafs), 2):
                l = leafs[i]
                if i+1>=len(leafs): r = '0x0'
                else              : r = leafs[i+1]
                parents.append(sha(l+r))
            leafs = parents
            self.mt[f'{len(self)+1}'] = parents
            parents = []
    
    @property
    def root(self)      : return self.mt[str(len(self))][0]
    def __eq__ (self, o): return self.root == o.root
    def __len__(self)   : return len(self.mt)
    def __getitem__(self, k): return self.mt[str(k)]
    def __str__(self):
        s = ''
        for k,v in self.mt.items(): 
            s += f'height {k}'
            for h in v: s += f'\n {ph(h)}'
            s += '\n'
        return s

Create Merkle Tree from `N` random txs.

In [19]:
N   = 8
txs = rtxs(N)

mt = MerkleTree(txs); print(mt)

height 1
 🔍 0xa7ad42f76e...283
 💽 0x574305c1ba...290
 📯 0x89f05c7d0c...90b
 💚 0x3486bbbf5f...6c6
 📫 0x85dbc46ef2...101
 💐 0x2a07fba863...370
 🔬 0xc6b5ea3bf7...913
 📃 0x5d6ddecbc7...8c2
height 2
 💡 0x3ba952f9cb...fc6
 📭 0x87dd9cf375...dc8
 🕅 0xdfd65392dc...476
 👬 0x0665c208ad...2f2
height 3
 🔮 0xc8e1bc105e...b36
 📽 0x97f6d6884e...ea7
height 4
 💕 0x2fa7341cff...18e



In [20]:
assert len(mt) == 4

The merkle tree root hash is the top most node. If anything in the tree changes the root changes as well.

In [21]:
ph(mt.root)

'💕 0x2fa7341cff...18e'

If we change something in any tx the whole Merkle Tree changes. Here we change the `value` of the third tx. Compare the hashes below with the ones above.

In [24]:
txs[2].value = 500

changed_mt = MerkleTree(txs); print(changed_mt)

height 1
 🔍 0xa7ad42f76e...283
 💽 0x574305c1ba...290
 📧 0x81028b118a...ee8
 💚 0x3486bbbf5f...6c6
 📫 0x85dbc46ef2...101
 💐 0x2a07fba863...370
 🔬 0xc6b5ea3bf7...913
 📃 0x5d6ddecbc7...8c2
height 2
 💡 0x3ba952f9cb...fc6
 💴 0x4e11233cf1...f7d
 🕅 0xdfd65392dc...476
 👬 0x0665c208ad...2f2
height 3
 🔥 0xbf5301066d...1b2
 📽 0x97f6d6884e...ea7
height 4
 👦 0x00d521e81f...ae5



In [25]:
assert mt != changed_mt

## Merkle Proof

Proof that `tx` at position `pos` in the merkle tree `mt` is part of it by using a corresponding merkle branch `mb`.

A merkle branch consists of those nodes that are needed to build up the root hash of the merkle tree. For example:

Merkle branch `[5,3,0, ..., x]` stands for the following: 
- node *5* from height *0*
- node *3* from height *1*
- node *0* from height *2*
- ...
- node *x* from height `len(mt)-1`

In [26]:
def proof(mt, mb, tx):
    if mb[0]%2 != 0: cn = sha(tx.hash     +mt[1][mb[0]])
    else           : cn = sha(mt[1][mb[0]]+tx.hash)
    for i in range(2, len(mt)):
        if mb[i-1]%2 != 0: cn = sha(cn            +mt[i][mb[i-1]])
        else             : cn = sha(mt[i][mb[i-1]]+cn)
    return cn == mt.root

Examples of some txs with their corresponding merkle branch.

In [30]:
tx = txs[0]
mb = [1,1,1]

assert proof(mt, mb, tx)

In [37]:
tx = txs[1]
mb = [0,1,1]

assert proof(mt, mb, tx)

In [36]:
tx = txs[4]
mb = [5,3,0]

assert proof(mt, mb, tx)